In [1]:
from dask import bag
from dask.diagnostics import ProgressBar
import pandas as pd, numpy as np
import os
import pathlib 
import re
import multiprocessing


In [2]:
DATA_PATH = pathlib.Path.cwd().parents[1] / "ArrestCaseDataPipeline/DCSC-Scraper"

In [3]:
# To subset by party, we need to get a list of parties. 

party_files = DATA_PATH.glob("CMD_party*.csv")
party_df = pd.concat((pd.read_csv(f) for f in party_files), ignore_index=True).drop(columns="Party_Index").drop_duplicates().reset_index(drop=True)
all_parties = party_df['party_label'].unique()


In [4]:
# Add code to remove detainees. 


In [5]:
# We'll take 1000 parties at at time, and that will give us about 4000 cases per partition

chunk_size = 1000
party_lists = [all_parties[i:i + chunk_size] for i in range(0, len(all_parties), chunk_size)]


In [6]:
# Now we partition the data. 

def partition(data_type, df, party_lists):

    for partition_number, party_list in enumerate(party_lists):
        case_list = party_df[party_df.party_label.isin(party_list)]['Case_ID'].drop_duplicates(keep='first')
        subset = df[df['Case_ID'].isin(case_list)]
        partition_path = pathlib.Path(pathlib.Path.cwd() , 'partitions' , 'p%s' % str(partition_number).zfill(2) )
        partition_path.mkdir(parents=True, exist_ok=True)
        subset.to_pickle(partition_path/pathlib.Path('%s.dat' % data_type), compression='gzip')

    
for data_type in ['party', 'summary', 'docket', 'receipts' ]:
    df_files = DATA_PATH.glob("CMD_%s*.csv" % data_type)
    df = pd.concat((pd.read_csv(f) for f in df_files), ignore_index=True)
    partition(data_type, df, party_lists)


In [7]:
print('done!')

done!
